In [2]:
import pandas as pd
import cleaningtools as ct 
import file_config as fconfig
import numpy as np
import math


check for pandas


In [3]:
file=fconfig.sample_file
data=pd.read_csv(file)
drop_index=data[data['From_ft'].isna()].index
data=data.drop(drop_index,axis=0)
data

,Unnamed: 0,Geo,Sample_ID,Hole_ID,From_ft,To_Ft,From_m,To_m,QAQC,Description,foldername,filename,Unnamed: 12,_2,_3
0,2,JL,G453729,KM-22-82,731.0,736.0,222.8088,224.3328,NaN,"serecite-quartz-chlorite schist, no mineraliza...",KM-22-82,KM-22-82 Drill Log.xlsx,NaN,NaN,NaN
1,4,JL,G453731,KM-22-82,736.0,741.0,224.3328,225.8568,NaN,"serecite-quartz-chlorite schist, no mineraliza...",KM-22-82,KM-22-82 Drill Log.xlsx,NaN,NaN,NaN
2,5,JL,G453732,KM-22-82,741.0,743.0,225.8568,226.4664,NaN,"serecite-quartz-chlorite schist, dolomite vein...",KM-22-82,KM-22-82 Drill Log.xlsx,NaN,NaN,NaN
3,6,JL,G453733,KM-22-82,743.0,745.0,226.4664,227.0760,NaN,zones of stringer to semi-massive sulfides in ...,KM-22-82,KM-22-82 Drill Log.xlsx,NaN,NaN,NaN
4,7,JL,G453734,KM-22-82,745.0,748.0,227.0760,227.9904,NaN,very weak stringer sulfides in serecite-quartz...,KM-22-82,KM-22-82 Drill Log.xlsx,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1338,1528,RH,C523556,KM-22-30,869.0,874.0,264.8700,266.4000,NaN,"silicified schist, 2-3% pyrite, 2% chalcopyrit...",KM-22-30,Kay Drill Log KM-21-30.xlsx,C523,556.0,C523556
1339,1529,RH,C523557,KM-22-30,874.0,879.0,266.4000,267.9200,NaN,"silicified schist, 1-2% pyrite, 0.1% chalcopyr...",KM-22-30,Kay Drill Log KM-21-30.xlsx,C523,557.0,C523557
1340,1530,RH,C523558,KM-22-30,879.0,883.0,267.9200,269.1400,NaN,"light gray schist, 1-2% pyrite, 0.1% chalcopyr...",KM-22-30,Kay Drill Log KM-21-30.xlsx,C523,558.0,C523558
1341,1531,RH,C523559,KM-22-30,883.0,886.0,269.1400,270.0500,NaN,"light gray schist, minor black chlorite, no mi...",KM-22-30,Kay Drill Log KM-21-30.xlsx,C523,559.0,C523559


In [9]:

zipped=zip(data['Sample_ID'], data['From_ft'], data['To_Ft'])
df=pd.DataFrame([(id, y) for id, start, end in zipped for y in np.arange(start, end,.5)],
                   columns=['sample_id','depth'])
df


,sample_id,depth
0,G453729,731.0
1,G453729,731.5
2,G453729,732.0
3,G453729,732.5
4,G453729,733.0
...,...,...
11250,C523561,886.5
11251,C523561,887.0
11252,C523561,887.5
11253,C523561,888.0


In [12]:
big_df=df.merge(data , left_on='sample_id' ,right_on ='Sample_ID')

In [13]:
big_df

,sample_id,depth,Unnamed: 0,Geo,Sample_ID,Hole_ID,From_ft,To_Ft,From_m,To_m,QAQC,Description,foldername,filename,Unnamed: 12,_2,_3
0,G453729,731.0,2,JL,G453729,KM-22-82,731.0,736.0,222.8088,224.3328,NaN,"serecite-quartz-chlorite schist, no mineraliza...",KM-22-82,KM-22-82 Drill Log.xlsx,NaN,NaN,NaN
1,G453729,731.5,2,JL,G453729,KM-22-82,731.0,736.0,222.8088,224.3328,NaN,"serecite-quartz-chlorite schist, no mineraliza...",KM-22-82,KM-22-82 Drill Log.xlsx,NaN,NaN,NaN
2,G453729,732.0,2,JL,G453729,KM-22-82,731.0,736.0,222.8088,224.3328,NaN,"serecite-quartz-chlorite schist, no mineraliza...",KM-22-82,KM-22-82 Drill Log.xlsx,NaN,NaN,NaN
3,G453729,732.5,2,JL,G453729,KM-22-82,731.0,736.0,222.8088,224.3328,NaN,"serecite-quartz-chlorite schist, no mineraliza...",KM-22-82,KM-22-82 Drill Log.xlsx,NaN,NaN,NaN
4,G453729,733.0,2,JL,G453729,KM-22-82,731.0,736.0,222.8088,224.3328,NaN,"serecite-quartz-chlorite schist, no mineraliza...",KM-22-82,KM-22-82 Drill Log.xlsx,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11250,C523561,886.5,1533,RH,C523561,KM-22-30,886.0,889.0,270.0500,270.9700,NaN,"light gray schist, moderate black chlorite, no...",KM-22-30,Kay Drill Log KM-21-30.xlsx,C523,561.0,C523561
11251,C523561,887.0,1533,RH,C523561,KM-22-30,886.0,889.0,270.0500,270.9700,NaN,"light gray schist, moderate black chlorite, no...",KM-22-30,Kay Drill Log KM-21-30.xlsx,C523,561.0,C523561
11252,C523561,887.5,1533,RH,C523561,KM-22-30,886.0,889.0,270.0500,270.9700,NaN,"light gray schist, moderate black chlorite, no...",KM-22-30,Kay Drill Log KM-21-30.xlsx,C523,561.0,C523561
11253,C523561,888.0,1533,RH,C523561,KM-22-30,886.0,889.0,270.0500,270.9700,NaN,"light gray schist, moderate black chlorite, no...",KM-22-30,Kay Drill Log KM-21-30.xlsx,C523,561.0,C523561


In [14]:
assay=pd.read_csv(fconfig.assay_file)
big_df=big_df.merge(assay,on='sample_id')

,sample_id,depth,Unnamed: 0,Geo,Sample_ID,Hole_ID,From_ft,To_Ft,From_m,To_m,...,Ta,Te,Y,Zr,Sn_2,Pb_2,start_depth,end_depth,file,folder
0,E571371,605.5,306,EK,E571371,KM-22-73,605.5,607.5,184.5564,185.1660,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TU22125500.csv,TU22125500 71A 72 73 63C
1,E571371,606.0,306,EK,E571371,KM-22-73,605.5,607.5,184.5564,185.1660,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TU22125500.csv,TU22125500 71A 72 73 63C
2,E571371,606.5,306,EK,E571371,KM-22-73,605.5,607.5,184.5564,185.1660,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TU22125500.csv,TU22125500 71A 72 73 63C
3,E571371,607.0,306,EK,E571371,KM-22-73,605.5,607.5,184.5564,185.1660,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TU22125500.csv,TU22125500 71A 72 73 63C
4,E571372,832.0,307,EK,E571372,KM-22-73,832.0,837.0,253.5936,255.1176,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TU22125500.csv,TU22125500 71A 72 73 63C
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13402,C523561,886.5,1533,RH,C523561,KM-22-30,886.0,889.0,270.0500,270.9700,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TU21197031.csv,TU21197031 29 30 31 32 33 34
13403,C523561,887.0,1533,RH,C523561,KM-22-30,886.0,889.0,270.0500,270.9700,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TU21197031.csv,TU21197031 29 30 31 32 33 34
13404,C523561,887.5,1533,RH,C523561,KM-22-30,886.0,889.0,270.0500,270.9700,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TU21197031.csv,TU21197031 29 30 31 32 33 34
13405,C523561,888.0,1533,RH,C523561,KM-22-30,886.0,889.0,270.0500,270.9700,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TU21197031.csv,TU21197031 29 30 31 32 33 34


In [15]:
assay=pd.read_csv(fconfig.assay_file)
big_df=big_df.merge(assay,on='sample_id')